# Regras

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

#### biblioteca usada para nineração de dados
- !pip install Orange3

In [2]:
import Orange

### Base risco de crédito

In [11]:
# Vamos cria uma variavel "base_risco_credito " e tomamos o fromato "Orange.data.Table()"
base_risco_credito = Orange.data.Table('risco_credito_regras.csv')

In [4]:
base_risco_credito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [13]:
#Obsevamos o dominio
base_risco_credito.domain

[historia, divida, garantias, renda | risco]

In [14]:
#Trabalhamos com algoritmos de regras de associação (criação de regras)
cn2 = Orange.classification.rules.CN2Learner()
regras_risco_credito = cn2(base_risco_credito)

In [15]:
# observamos as regras geradas
for regras in regras_risco_credito.rule_list:
  print(regras)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=alto 
IF TRUE THEN risco=alto 


In [8]:
# história boa, dívida alta, garantias nenhuma, renda > 35
# história ruim, dívida alta, garantias adequada, renda < 15
previsoes = regras_risco_credito([['boa', 'alta', 'nenhuma', 'acima_35'], ['ruim', 'alta', 'adequada', '0_15']])
previsoes

array([1, 0])

In [9]:
base_risco_credito.domain.class_var.values

('alto', 'baixo', 'moderado')

In [16]:
#mostramos os label
for i in previsoes:
  #print(i)
  print(base_risco_credito.domain.class_var.values[i])

baixo
alto


## Base credit data

In [17]:
base_credit = Orange.data.Table('credit_data_regras.csv')

In [18]:
base_credit.domain

[income, age, loan | default]

In [30]:
#criamos uma nova variavel
base_dividida = Orange.evaluation.testing.sample(base_credit, n = 0.25)

In [20]:
base_dividida

([[43069.7, 41.24, 4091.56 | 0],
  [35145.1, 47.7934, 510.74 | 0],
  [33197.8, 59.4868, 3993.15 | 0],
  [56253, 57.2783, 7327.07 | 0],
  [31112.1, 49.616, 424.352 | 0],
  ...
 ],
 [[39703.8, 47.4687, 2403.48 | 0],
  [59514, 57.0178, 2504.72 | 0],
  [59253.1, 36.9204, 7327.28 | 0],
  [26039, 49.3904, 2056.75 | 0],
  [66236.9, 35.9191, 2756.97 | 0],
  ...
 ])

In [21]:
base_dividida[0]

[[43069.7, 41.24, 4091.56 | 0],
 [35145.1, 47.7934, 510.74 | 0],
 [33197.8, 59.4868, 3993.15 | 0],
 [56253, 57.2783, 7327.07 | 0],
 [31112.1, 49.616, 424.352 | 0],
 ...
]

In [22]:
base_dividida[1]

[[39703.8, 47.4687, 2403.48 | 0],
 [59514, 57.0178, 2504.72 | 0],
 [59253.1, 36.9204, 7327.28 | 0],
 [26039, 49.3904, 2056.75 | 0],
 [66236.9, 35.9191, 2756.97 | 0],
 ...
]

In [23]:
base_treinamento = base_dividida[1]
base_teste = base_dividida[0]

In [24]:
len(base_treinamento), len(base_teste)

(1500, 500)

In [25]:
cn2 = Orange.classification.rules.CN2Learner()
regras_credit = cn2(base_treinamento)

In [31]:
#visualização das regras
for regras in regras_credit.rule_list:
  print(regras)

IF age>=34.9257164876908 THEN default=0 
IF age>=34.7514377613552 THEN default=1 
IF loan<=2495.13299137587 AND income>=20145.9885970689 THEN default=0 
IF loan>=7718.479795185201 AND age>=18.1760434475727 THEN default=1 
IF income>=58147.7998569608 THEN default=0 
IF loan>=5836.56338145928 AND age>=29.4230185477063 THEN default=1 
IF loan>=6415.0862444378 THEN default=1 
IF age>=33.9493660898879 AND income>=34237.5754192472 THEN default=0 
IF income>=38129.754869733 AND income>=50360.6787929311 THEN default=0 
IF loan<=3682.8619314122707 AND income>=29724.4768752716 THEN default=0 
IF loan>=5836.56338145928 THEN default=1 
IF loan>=5722.58198121271 THEN default=0 
IF income>=42236.4560933784 AND income>=45360.716274008206 THEN default=0 
IF loan>=5193.83819663771 THEN default=1 
IF loan>=5154.909842680399 THEN default=0 
IF loan>=5043.14863656084 THEN default=1 
IF loan>=5036.25528047363 THEN default=0 
IF loan>=4849.33378471958 THEN default=1 
IF age>=29.732926420401803 AND income>=2

In [27]:
previsoes = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [lambda testdata: regras_credit])

In [28]:
previsoes

In [29]:
Orange.evaluation.CA(previsoes)

array([0.984])

### Base census - 78.90% (executado na interface gráfica do Orange)

- https://orangedatamining.com/download/#linux

## Classificador base - Majority learner

In [32]:
base_credit = Orange.data.Table('credit_data_regras.csv')

In [33]:
base_credit.domain

[income, age, loan | default]

In [34]:
majority = Orange.classification.MajorityLearner()

In [35]:
previsoes = Orange.evaluation.testing.TestOnTestData(base_credit, base_credit, [majority])

In [36]:
Orange.evaluation.CA(previsoes)

array([0.8585])

In [41]:
for registro in base_credit:
  print(registro.get_class())

0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
1
0
0
1
0
0
0
0


In [47]:
from collections import Counter
Counter(str(registro.get_class()) for registro in base_credit)

Counter({'0': 1717, '1': 283})

In [48]:
1717 / 2000

0.8585

## Base census - 75.91%

In [42]:
base_census = Orange.data.Table('census_regras.csv')

In [43]:
base_census.domain

[age, workclass, final-weight, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loos, hour-per-week, native-country | income]

In [49]:
majority = Orange.classification.MajorityLearner()
previsoes = Orange.evaluation.testing.TestOnTestData(base_census, base_census, [majority])
Orange.evaluation.CA(previsoes)

array([0.75919044])

In [50]:
Counter(str(registro.get_class()) for registro in base_census)

Counter({'<=50K': 24720, '>50K': 7841})

In [51]:
24720 / (24720 + 7841)

0.7591904425539756

## Fin deste tema